# Análisis de Componentes Principales: Países del Mundo

En  este notebook vamos a proceder al cálculo de las componentes principales para el dataset con información de los distintos países que ya hemos comenzado a explorar en el notebook anterior. Una vez extraídas las componentes principales graficaremos los datos intentando ver si esto nos aporta alguna información o resulta inútil. 

<div>
<img src="./media/mapamundi.jpg" width="500"/>
</div>

Comenzamos importando los módulos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Comenzamos cargando los datos en un dataframe:

In [ ]:
paises_data = pd.read_csv('./data/country_data.csv', index_col = 0)

Como siempre nos aseguramos de que los datos se hayan cargado correctamente:

In [ ]:
paises_data.head()

In [ ]:
paises_data.shape

De nuevo observamos que existen muchas diferencias entre las escalas de las variables por lo que pasamos como siempre a escalar los datos:

In [ ]:
from sklearn.preprocessing import scale
X_paises = pd.DataFrame(scale(paises_data), index=paises_data.index, columns=paises_data.columns)

Con esto hemos generado nuestros datos escalados:

In [ ]:
X_paises.head()

Una vez hecho esto podemos comenzar calculando nuestros vectores de carga:

In [ ]:
from sklearn.decomposition import PCA
pca_loadings = pd.DataFrame(PCA().fit(X_paises).components_.T, index=X_paises.columns, columns=['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9'])
pca_loadings

Observamos la proyección de cada variable en cada vector de carga. Tenemos en total 9 vectores de carga, ergo, 9 componentes pricipales. Esto era de esperar pues si recordamos nuestro dataframe tiene 167 observaciones y 9 variables luego tendremos un total de min (167-1,9) componentes principales.

Podemos construir una tabla en la que vemos como se proyectan los países sobre las nuevas coordenadas:

In [ ]:
pca = PCA()
df_plot = pd.DataFrame(pca.fit_transform(X_paises), columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9'], index=X_paises.index)
df_plot.head()

Una vez calculadas las componentes podemos visualizar la información sobre las dos primeras:

In [ ]:
fig , ax1 = plt.subplots(figsize=(30,15))
ax1.set_xlim(-3.5,3.5)
ax1.set_ylim(-3.5,3.5)
# Representamos las componentes principales
for i in df_plot.index:
    ax1.annotate(i, (-df_plot.PC1.loc[i], -df_plot.PC2.loc[i]), ha='center')
# Representamos las rectas de referencia
ax1.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.set_xlabel('Primera Componente Principal')
ax1.set_ylabel('Segunda Componente Principal')
# Construimos los ejes para los vectores de carga
ax2 = ax1.twinx().twiny()
ax2.set_ylim(-1,1)
ax2.set_xlim(-1,1)
ax2.set_xlabel('En rojo: Vectores de carga', color='red')
# Etiquetamos los vectores de carga
a = 1.07
for i in pca_loadings[['V1', 'V2']].index:
    ax2.annotate(i, (-pca_loadings.V1.loc[i]*a, -pca_loadings.V2.loc[i]*a), color='red')
# Representamos dichos vectores
ax2.arrow(0,0,-pca_loadings.V1[0], -pca_loadings.V2[0])
ax2.arrow(0,0,-pca_loadings.V1[1], -pca_loadings.V2[1])
ax2.arrow(0,0,-pca_loadings.V1[2], -pca_loadings.V2[2])
ax2.arrow(0,0,-pca_loadings.V1[3], -pca_loadings.V2[3])
ax2.arrow(0,0,-pca_loadings.V1[4], -pca_loadings.V2[4])
ax2.arrow(0,0,-pca_loadings.V1[5], -pca_loadings.V2[5])
ax2.arrow(0,0,-pca_loadings.V1[6], -pca_loadings.V2[6])
ax2.arrow(0,0,-pca_loadings.V1[7], -pca_loadings.V2[7])
ax2.arrow(0,0,-pca_loadings.V1[8], -pca_loadings.V2[8]);

**Hipótesis**

* Solo con dos componentes ya podemos sacar algunas conclusiones interesantes como que los países que solemos denominar desarrollados toman valores negativos respecto a la primera componente principal y positivos respecto a la segunda. Podemos observar como aparecen agrupados en la esquina superior izquierda.

* Las variables que más influyen a la primera componente (aquellas que se representan con vectores más horizontales) son las económicas con los ingresos, el PIB y la inflación. 

* Las variables que más influyen a la segunda componente (aquellas que se representan con vectores más verticales) son la exportación e importación y el gasto en sanidad.

* A medida que avanzamos por el eje horizontal hacia la derecha observamos como se distinguen algunos países poco desarrolados y con peores condiciones de vida como pueden ser Mali, Burkina Faso o Sierra Leona.

Para evaluar la calidad de este modelo podemos comprobar cuánta varianza se explica en cada componente:

In [ ]:
pca.explained_variance_

El número fuera de contexto puede resultar confuso por lo que observamos en su lugar el ratio de varianza explicada:

In [ ]:
pca.explained_variance_ratio_

Puede resultar interesante también observar la suma acumulada de estos ratios:

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

Observamos que con dos componentes expresamos un 63% de la información, una cantidad nada desdeñable. Esto valida en parte las hipótesis presentadas previamente. 

Si deseamos fijar un número de componentes principales podemos, como vimos en la teoría, construir los gráficos de sedimentación:

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4,5,6,7,8,9], pca.explained_variance_ratio_, '-o')
plt.ylabel('Proporción de varianza explicada')
plt.xlabel('Componentes Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4,5,6,7,8,9]);

Personalmente a la hora de elegir el número de componentes siempre me ha parecido más intuitivo el gráfico de varianza acumulada:

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4,5,6,7,8,9], np.cumsum(pca.explained_variance_ratio_), '-s')
plt.ylabel('Proporción de varianza explicada acumulada')
plt.xlabel('Componentes Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4,5,6,7,8,9]);

Una vez construidos los gráficos tendríamos que decidir con cuántas componentes nos quedamos, todo depende del problema que enfoquemos y la importancia de la precisión vs explicabilidad.

En los gráficos observamos que a partir de la tercera componente la curva se va aplanando poco a poco y que a partir de la sexta las nuevas componentes prácticamente no añaden información. Así si deseamos visualizar nos quedamos con dos pero si queremos una mayor precisión, por ejemplo, para aplicar posteriomente un método supervisado elegiríamos un total de 4 o 5 componentes.